# Imports

In [32]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import math
import numpy as np

pd.set_option('display.max_columns', None)

In [2]:
import pandas as pd
import sys
import os

# Add the src directory to the path so we can import our modules
sys.path.append(os.path.abspath('..'))

# Import the utility function for one-time directory change
from src.utilities.path_utilities import ensure_parent_dir

# Change to parent directory (only happens once even if cell is re-executed)
ensure_parent_dir('notebook_setup')

Original directory: /Users/adrianhajdukiewicz/projects/private/2025_data_biz_hackathon/notebooks
Changed directory to: /Users/adrianhajdukiewicz/projects/private/2025_data_biz_hackathon


'/Users/adrianhajdukiewicz/projects/private/2025_data_biz_hackathon'

In [3]:
pois = gpd.read_file("data/mazowieckie_all/gis_osm_pois_free_1.shp", encoding="utf-8")

In [4]:
pois

,osm_id,code,fclass,name,geometry
0,30776210,2301,restaurant,Dziupla,POINT (20.76933 52.3084)
1,31005854,2301,restaurant,None,POINT (20.94595 52.17691)
2,31156693,2502,bakery,Galeria Wypieków Lubaszka,POINT (20.95489 52.271)
3,31917380,2724,memorial,Pomnik Tysiąclecia Jazdy Polskiej,POINT (21.01451 52.21653)
4,38202108,2724,memorial,Pomnik polskich kryptologów,POINT (21.03313 52.12883)
...,...,...,...,...,...
98215,12847077865,2902,bench,None,POINT (21.11404 51.3935)
98216,12847077866,2902,bench,None,POINT (21.11415 51.39326)
98217,12847516473,2033,recycling_clothes,None,POINT (21.02689 52.18346)
98218,12847549932,2421,shelter,None,POINT (20.45471 52.28655)


In [5]:
pois[pois["name"]=="Żabka"]

,osm_id,code,fclass,name,geometry
96,260458573,2511,convenience,Żabka,POINT (21.04185 52.14717)
119,270276669,2511,convenience,Żabka,POINT (21.10012 52.23946)
176,280726070,2511,convenience,Żabka,POINT (21.00357 52.22635)
198,293472005,2511,convenience,Żabka,POINT (21.03772 52.16166)
247,310280465,2511,convenience,Żabka,POINT (21.00146 52.22715)
...,...,...,...,...,...
96900,12766655646,2511,convenience,Żabka,POINT (19.6884 52.5435)
96905,12766948609,2511,convenience,Żabka,POINT (20.61935 52.1205)
97380,12788219468,2511,convenience,Żabka,POINT (22.27099 52.16538)
97556,12803070865,2511,convenience,Żabka,POINT (19.80433 52.67157)


In [6]:
pois_a = gpd.read_file("data/mazowieckie_all/gis_osm_pois_a_free_1.shp", encoding="utf-8")

In [7]:
pois_a = pois_a.to_crs(epsg=2180)

In [8]:
pois_a["centroid"] = pois_a.geometry.centroid

In [9]:
pois_a["geometry"] = pois_a["centroid"].to_crs(epsg=4326)

In [10]:
pois_a = pois_a.drop(columns=["centroid"])

In [11]:
result = gpd.GeoDataFrame(pd.concat([pois, pois_a], ignore_index=True), geometry='geometry')

In [12]:
result

,osm_id,code,fclass,name,geometry
0,30776210,2301,restaurant,Dziupla,POINT (20.76933 52.3084)
1,31005854,2301,restaurant,None,POINT (20.94595 52.17691)
2,31156693,2502,bakery,Galeria Wypieków Lubaszka,POINT (20.95489 52.271)
3,31917380,2724,memorial,Pomnik Tysiąclecia Jazdy Polskiej,POINT (21.01451 52.21653)
4,38202108,2724,memorial,Pomnik polskich kryptologów,POINT (21.03313 52.12883)
...,...,...,...,...,...
136111,1387801358,2252,pitch,None,POINT (21.25041 51.4655)
136112,1387801359,2252,pitch,None,POINT (21.25087 51.46521)
136113,1387801361,2082,school,Publiczna Szkoła Podstawowa im. Tadeusza Kości...,POINT (21.25029 51.46558)
136114,1387818672,2421,shelter,None,POINT (20.97839 51.39068)


In [13]:
result["lon"] = result.geometry.x
result["lat"] = result.geometry.y
result_clean = result[["fclass", "lon", "lat"]]

# convert geodataframe to pandas dataframe
result_clean = pd.DataFrame(result_clean)


In [14]:
result_clean.head()

,fclass,lon,lat
0,restaurant,20.769334,52.308402
1,restaurant,20.945950,52.176908
2,bakery,20.954893,52.271002
3,memorial,21.014506,52.216532
4,memorial,21.033132,52.128829


# Join Zabka's data

In [15]:
y_raw = pd.read_csv('data/y_raw_before_counting_per_square.csv')

y_raw['fclass'] = 'target'
y_raw = y_raw.rename(columns={"latitude": "lat", "longitude": "lon"})


# Add 'fclass' named 'y' to the result_clean dataframe
result_clean = pd.concat([result_clean, y_raw[['fclass', 'lat', 'lon']]])

In [16]:
result_clean

,fclass,lon,lat
0,restaurant,20.769334,52.308402
1,restaurant,20.945950,52.176908
2,bakery,20.954893,52.271002
3,memorial,21.014506,52.216532
4,memorial,21.033132,52.128829
...,...,...,...
42207,target,18.183524,50.336754
42208,target,17.615581,50.751824
42209,target,18.791962,51.976362
42210,target,21.081131,52.274640


In [17]:
result_clean.head()

,fclass,lon,lat
0,restaurant,20.769334,52.308402
1,restaurant,20.945950,52.176908
2,bakery,20.954893,52.271002
3,memorial,21.014506,52.216532
4,memorial,21.033132,52.128829


In [18]:
result_clean.columns

Index(['fclass', 'lon', 'lat'], dtype='object')

In [19]:

result_clean

,fclass,lon,lat
0,restaurant,20.769334,52.308402
1,restaurant,20.945950,52.176908
2,bakery,20.954893,52.271002
3,memorial,21.014506,52.216532
4,memorial,21.033132,52.128829
...,...,...,...
42207,target,18.183524,50.336754
42208,target,17.615581,50.751824
42209,target,18.791962,51.976362
42210,target,21.081131,52.274640


In [20]:
# # Load the y_raw_before_counting_per_square.csv file
# y_raw = pd.read_csv('data/y_raw_before_counting_per_square.csv')
# y_raw['fclass'] = 'target'

# y_raw = y_raw.rename(columns={"latitude": "lat", "longitude": "lon"})

# # Add 'fclass' named 'y' to the result_clean dataframe
# result_clean = pd.concat([result_clean, y_raw])
# result_clean["counter"] = 1
# # Show the result
# result_clean.tail()

In [21]:
result_clean["x"] = np.floor((result_clean["lon"] - 52.1147) / 0.005)
result_clean["y"] = np.floor((result_clean["lat"] - 20.7622) / 0.01)
result_clean["key"] = result_clean["x"].astype(str) + "_" + result_clean["y"].astype(str)

In [22]:
result_clean['counter'] = 1

In [23]:
result_clean.tail()

,fclass,lon,lat,x,y,key,counter
42207,target,18.183524,50.336754,-6787.0,2957.0,-6787.0_2957.0,1
42208,target,17.615581,50.751824,-6900.0,2998.0,-6900.0_2998.0,1
42209,target,18.791962,51.976362,-6665.0,3121.0,-6665.0_3121.0,1
42210,target,21.081131,52.274640,-6207.0,3151.0,-6207.0_3151.0,1
42211,target,18.603009,53.052142,-6703.0,3228.0,-6703.0_3228.0,1


In [24]:
# filter key value == '-10417.0_-2087.0'
#result_clean = result_clean[result_clean["key"] != "-10417.0_-2087.0"]



In [25]:
result_pivoted = result_clean.pivot_table(
    index="key",      # wiersze
    columns="fclass",         # kolumny
    values="counter",        # co zliczamy
    aggfunc="count",          # funkcja agregująca
    #fill_value=0              # zamiast NaN pokaż 0
)
result_pivoted

fclass,archaeological,arts_centre,artwork,atm,attraction,bakery,bank,bar,battlefield,beauty_shop,bench,beverages,bicycle_rental,bicycle_shop,biergarten,bookshop,butcher,cafe,camera_surveillance,camp_site,car_dealership,car_rental,car_wash,caravan_site,castle,chalet,chemist,cinema,clinic,clothes,college,comms_tower,community_centre,computer_shop,convenience,courthouse,dentist,department_store,doctors,dog_park,doityourself,drinking_water,embassy,fast_food,fire_station,florist,food_court,fort,fountain,furniture_shop,garden_centre,general,gift_shop,golf_course,graveyard,greengrocer,guesthouse,hairdresser,hospital,hostel,hotel,hunting_stand,ice_rink,jeweller,kindergarten,kiosk,laundry,library,mall,market_place,memorial,mobile_phone_shop,monument,motel,museum,newsagent,nightclub,nursing_home,observation_tower,optician,outdoor_shop,park,pharmacy,picnic_site,pitch,playground,police,post_box,post_office,prison,pub,public_building,recycling,recycling_clothes,recycling_glass,recycling_metal,recycling_paper,restaurant,ruins,school,shelter,shoe_shop,sports_centre,sports_shop,stadium,stationery,supermarket,swimming_pool,target,telephone,theatre,theme_park,toilet,tourist_info,tower,town_hall,toy_shop,track,travel_agent,university,vending_any,vending_machine,vending_parking,veterinary,video_shop,viewpoint,waste_basket,wastewater_plant,water_mill,water_tower,water_well,water_works,wayside_cross,wayside_shrine,windmill,zoo
key,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
-10223.0_-1977.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-10423.0_-2077.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,733.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-5611.0_2991.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-5614.0_3013.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-5616.0_2982.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [26]:
# filter rows where bakery col is greater than 0
result_pivoted[result_pivoted['bank'] > 0]

fclass,archaeological,arts_centre,artwork,atm,attraction,bakery,bank,bar,battlefield,beauty_shop,bench,beverages,bicycle_rental,bicycle_shop,biergarten,bookshop,butcher,cafe,camera_surveillance,camp_site,car_dealership,car_rental,car_wash,caravan_site,castle,chalet,chemist,cinema,clinic,clothes,college,comms_tower,community_centre,computer_shop,convenience,courthouse,dentist,department_store,doctors,dog_park,doityourself,drinking_water,embassy,fast_food,fire_station,florist,food_court,fort,fountain,furniture_shop,garden_centre,general,gift_shop,golf_course,graveyard,greengrocer,guesthouse,hairdresser,hospital,hostel,hotel,hunting_stand,ice_rink,jeweller,kindergarten,kiosk,laundry,library,mall,market_place,memorial,mobile_phone_shop,monument,motel,museum,newsagent,nightclub,nursing_home,observation_tower,optician,outdoor_shop,park,pharmacy,picnic_site,pitch,playground,police,post_box,post_office,prison,pub,public_building,recycling,recycling_clothes,recycling_glass,recycling_metal,recycling_paper,restaurant,ruins,school,shelter,shoe_shop,sports_centre,sports_shop,stadium,stationery,supermarket,swimming_pool,target,telephone,theatre,theme_park,toilet,tourist_info,tower,town_hall,toy_shop,track,travel_agent,university,vending_any,vending_machine,vending_parking,veterinary,video_shop,viewpoint,waste_basket,wastewater_plant,water_mill,water_tower,water_well,water_works,wayside_cross,wayside_shrine,windmill,zoo
key,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
-5814.0_3156.0,NaN,NaN,1.0,1.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,1.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
-5837.0_3142.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-5859.0_3154.0,NaN,NaN,NaN,1.0,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-5879.0_3144.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
-5880.0_3144.0,NaN,NaN,NaN,2.0,NaN,NaN,2.0,NaN,NaN,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,3.0,NaN,1.0,NaN,2.0,NaN,NaN,4.0,NaN,NaN,NaN

In [27]:
# sort by key asc
result_pivoted.sort_values(by='target')



fclass,archaeological,arts_centre,artwork,atm,attraction,bakery,bank,bar,battlefield,beauty_shop,bench,beverages,bicycle_rental,bicycle_shop,biergarten,bookshop,butcher,cafe,camera_surveillance,camp_site,car_dealership,car_rental,car_wash,caravan_site,castle,chalet,chemist,cinema,clinic,clothes,college,comms_tower,community_centre,computer_shop,convenience,courthouse,dentist,department_store,doctors,dog_park,doityourself,drinking_water,embassy,fast_food,fire_station,florist,food_court,fort,fountain,furniture_shop,garden_centre,general,gift_shop,golf_course,graveyard,greengrocer,guesthouse,hairdresser,hospital,hostel,hotel,hunting_stand,ice_rink,jeweller,kindergarten,kiosk,laundry,library,mall,market_place,memorial,mobile_phone_shop,monument,motel,museum,newsagent,nightclub,nursing_home,observation_tower,optician,outdoor_shop,park,pharmacy,picnic_site,pitch,playground,police,post_box,post_office,prison,pub,public_building,recycling,recycling_clothes,recycling_glass,recycling_metal,recycling_paper,restaurant,ruins,school,shelter,shoe_shop,sports_centre,sports_shop,stadium,stationery,supermarket,swimming_pool,target,telephone,theatre,theme_park,toilet,tourist_info,tower,town_hall,toy_shop,track,travel_agent,university,vending_any,vending_machine,vending_parking,veterinary,video_shop,viewpoint,waste_basket,wastewater_plant,water_mill,water_tower,water_well,water_works,wayside_cross,wayside_shrine,windmill,zoo
key,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
-7174.0_3316.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-6550.0_3100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-6550.0_3101.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-6550.0_3102.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-6550.0_3115.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [28]:
# fill na with 0
result_pivoted = result_pivoted.fillna(0)


In [29]:
# filter rows where bakery col is greater than 0
result_pivoted[result_pivoted['bank'] > 0]

fclass,archaeological,arts_centre,artwork,atm,attraction,bakery,bank,bar,battlefield,beauty_shop,bench,beverages,bicycle_rental,bicycle_shop,biergarten,bookshop,butcher,cafe,camera_surveillance,camp_site,car_dealership,car_rental,car_wash,caravan_site,castle,chalet,chemist,cinema,clinic,clothes,college,comms_tower,community_centre,computer_shop,convenience,courthouse,dentist,department_store,doctors,dog_park,doityourself,drinking_water,embassy,fast_food,fire_station,florist,food_court,fort,fountain,furniture_shop,garden_centre,general,gift_shop,golf_course,graveyard,greengrocer,guesthouse,hairdresser,hospital,hostel,hotel,hunting_stand,ice_rink,jeweller,kindergarten,kiosk,laundry,library,mall,market_place,memorial,mobile_phone_shop,monument,motel,museum,newsagent,nightclub,nursing_home,observation_tower,optician,outdoor_shop,park,pharmacy,picnic_site,pitch,playground,police,post_box,post_office,prison,pub,public_building,recycling,recycling_clothes,recycling_glass,recycling_metal,recycling_paper,restaurant,ruins,school,shelter,shoe_shop,sports_centre,sports_shop,stadium,stationery,supermarket,swimming_pool,target,telephone,theatre,theme_park,toilet,tourist_info,tower,town_hall,toy_shop,track,travel_agent,university,vending_any,vending_machine,vending_parking,veterinary,video_shop,viewpoint,waste_basket,wastewater_plant,water_mill,water_tower,water_well,water_works,wayside_cross,wayside_shrine,windmill,zoo
key,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
-5814.0_3156.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
-5837.0_3142.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-5859.0_3154.0,0.0,0.0,0.0,1.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-5879.0_3144.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
-5880.0_3144.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,3.0,0.0,1.0,0.0,2.0,0.0,0.0,4.0,0.0,0.0,0.0

In [30]:
# convert index to column
result_pivoted = result_pivoted.reset_index()
result_pivoted
# drop value 'nan_nan' in key column
result_pivoted = result_pivoted[result_pivoted["key"] != "nan_nan"]
result_pivoted

fclass,key,archaeological,arts_centre,artwork,atm,attraction,bakery,bank,bar,battlefield,beauty_shop,bench,beverages,bicycle_rental,bicycle_shop,biergarten,bookshop,butcher,cafe,camera_surveillance,camp_site,car_dealership,car_rental,car_wash,caravan_site,castle,chalet,chemist,cinema,clinic,clothes,college,comms_tower,community_centre,computer_shop,convenience,courthouse,dentist,department_store,doctors,dog_park,doityourself,drinking_water,embassy,fast_food,fire_station,florist,food_court,fort,fountain,furniture_shop,garden_centre,general,gift_shop,golf_course,graveyard,greengrocer,guesthouse,hairdresser,hospital,hostel,hotel,hunting_stand,ice_rink,jeweller,kindergarten,kiosk,laundry,library,mall,market_place,memorial,mobile_phone_shop,monument,motel,museum,newsagent,nightclub,nursing_home,observation_tower,optician,outdoor_shop,park,pharmacy,picnic_site,pitch,playground,police,post_box,post_office,prison,pub,public_building,recycling,recycling_clothes,recycling_glass,recycling_metal,recycling_paper,restaurant,ruins,school,shelter,shoe_shop,sports_centre,sports_shop,stadium,stationery,supermarket,swimming_pool,target,telephone,theatre,theme_park,toilet,tourist_info,tower,town_hall,toy_shop,track,travel_agent,university,vending_any,vending_machine,vending_parking,veterinary,video_shop,viewpoint,waste_basket,wastewater_plant,water_mill,water_tower,water_well,water_works,wayside_cross,wayside_shrine,windmill,zoo
0,-10223.0_-1977.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-10423.0_-2077.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,733.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-5611.0_2991.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-5614.0_3013.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-5616.0_2982.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [31]:
result_pivoted.to_csv("data/y_and_x_ready_for_training.csv")